In [ ]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time
import numpy as np
import rotation_matrix as rm
import pandas as pd

In [ ]:
def connect_robot(ip="192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip)  # IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set


rtde_c, rtde_r, rtde_io_set = connect_robot()

In [ ]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(10)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")
gripper.move(45)

In [ ]:
# open csv file
test_bottles = pd.read_csv(
    "../pouring_simulation/output/summary_medium_test_50_final.csv"
)

In [ ]:
# print first 50 rows
test_bottles[0:50]

## Manually move robot to start position and read the position

In [ ]:
start_pos_horizontal = rtde_r.getActualTCPPose()

In [ ]:
start_pos_horizontal = [
    -0.05483472191563013 + 0.001,
    0.5215426516767572,
    0.16428408481957893 + 0.005,
    -1.5709891478285878,
    0.035603245728593275,
    0.03565316928565675,
]

In [ ]:
rtde_c.moveL(start_pos_horizontal, 0.1, 0.1)

In [ ]:
init_q = rtde_r.getActualQ()
init_q[5] += 0.25307274
rtde_c.moveJ(init_q, 0.1, 0.1)

start_pos_bottle_rotated = rtde_r.getActualTCPPose()

In [ ]:
gripper.move(45)

### Move to picking up position

In [ ]:
pick_up_pos_1 = [
    -0.1561726268124451,
    0.4693830793589494,
    0.16731695527821777,
    -1.558506867497785,
    0.2342495243576528,
    0.23421225149813182,
]

pick_up_pos_2 = [
    -0.22933701280059826,
    0.42583909934468755,
    0.1673295036223262,
    -1.5708309023061153,
    0.03953469319866688,
    0.03952195222219103,
]

pick_up_pos_3 = [
    -0.23469761648657117,
    0.4084118557626825,
    0.05347985664359268,
    -1.5711086518096042,
    0.017658218613486508,
    0.017625165019895223,
]

pick_up_pos_4 = [
    -0.23473853377504333,
    0.47152953670488773,
    0.053494416798714306,
    -1.5710858755788153,
    0.017660070482072698,
    0.01776629011841677,
]  # gripping_pos

move_to_pouring_pos = [
    -0.23469605422239773,
    0.471535779062463,
    0.1569813130298823,
    -1.5711022931402765,
    0.01762701939000535,
    0.017644393855374373,
]

In [ ]:
rtde_c.moveL(pick_up_pos_1, 0.3, 0.3)
rtde_c.moveL(pick_up_pos_2, 0.3, 0.3)
rtde_c.moveL(pick_up_pos_3, 0.3, 0.3)
time.sleep(2)
gripper.open()
rtde_c.moveL(pick_up_pos_4, 0.3, 0.3)

In [ ]:
def pick_up_flask():
    rtde_c.moveL(pick_up_pos_4, 0.3, 0.3)
    gripper.close()
    rtde_c.moveL(move_to_pouring_pos, 0.5, 0.5)
    rtde_c.moveL(start_pos_bottle_rotated, 0.3, 0.3)


def place_flask():
    rtde_c.moveL(move_to_pouring_pos, 0.1, 0.1)
    rtde_c.moveL(pick_up_pos_4, 0.3, 0.3)
    time.sleep(2)

# Start here:

## Input sample number:

In [ ]:
sample_number = 38

### Loading TCP file for movement

In [ ]:
scene_path = test_bottles.iloc[sample_number]["path"]
# delete first five characters of the path
scene_path = scene_path[5:]
scene_path = "../pouring_simulation/" + scene_path + "/TCP.txt"
# data_points = np.loadtxt(scene_path, delimiter=',', skiprows=1) # skiprows 1 and 1614

# Count the total number of rows in the file
with open(scene_path, "r") as file:
    num_rows = sum(1 for _ in file)
# load datapoints and skip first and last 5
data_points = np.loadtxt(scene_path, delimiter=",", skiprows=1, max_rows=num_rows - 2)

# convert from inches to meters
data_points[:, 0] = data_points[:, 0] * 0.0254
data_points[:, 1] = data_points[:, 1] * 0.0254
data_points[:, 2] = data_points[:, 2] + 0.25307274

# create list of positions
positions = []
for i in range(data_points.shape[0]):
    # positions.append([-data_points[i,1], -data_points[i,0], 0.0, 0.0, 0.0, data_points[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups
    positions.append(
        [data_points[i, 0], -data_points[i, 1], 0.0, 0.0, 0.0, data_points[i, 2]]
    )  # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

positions_converted = []
for i in range(data_points.shape[0]):
    # if none of the data entries is 0
    if not (positions[i][0] == 0 or positions[i][1] == 0 or positions[i][5] == 0):
        positions_converted.append(
            rm.PoseTrans(start_pos_horizontal, positions[i])
        )  # transform from tool coordinate system to base coordinate system

### Calculate speed


In [ ]:
# get first 5 x and z positions
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions) - 1):
    distances.append(np.linalg.norm(np.subtract(positions[i + 1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i] * 150)
    # speed.append(distances[i]*89)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

avg_speed = 0.007386997722436739

### Calculate pause time and find pause point

In [ ]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif (
        positions_converted[i]
        == positions_converted[i - 1]
        == positions_converted[i - 2]
        == positions_converted[i - 3]
        == positions_converted[i - 4]
    ):
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

### Show pouring settings from simulation

In [ ]:
print("Start Volume: ", test_bottles.iloc[sample_number]["volume_start"], "mL")
print(
    "Predicted Poured Volume: ", test_bottles.iloc[sample_number]["volume_poured"], "mL"
)
print(
    "Predicted Remaining Volume: ",
    float(test_bottles.iloc[sample_number]["volume_start"])
    - float(test_bottles.iloc[sample_number]["volume_poured"]),
    "mL",
)
print(
    "Predicted Received Volume: ",
    test_bottles.iloc[sample_number]["volume_received"],
    "mL",
)
print(
    "Predicted Spilled Volume: ",
    test_bottles.iloc[sample_number]["spilled_volume"],
    "mL",
)
print("\n")
print("Stop Angle: ", test_bottles.iloc[sample_number]["stop_angle"], "degrees")
print("Stop Time: ", test_bottles.iloc[sample_number]["pause_time"], "seconds")

### Do pouring

In [ ]:
time.sleep(2)

pick_up_flask()

velocity = avg_speed  # 0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.0015
blend_3 = 0.0
# path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
# path.append(path_pose1)
for i in range(len(positions_converted1) - 1):
    path.append(
        [
            positions_converted1[i][0],
            positions_converted1[i][1],
            positions_converted1[i][2],
            positions_converted1[i][3],
            positions_converted1[i][4],
            positions_converted1[i][5],
            velocity,
            acceleration,
            blend_i,
        ]
    )

path.append(
    [
        positions_converted1[-1][0],
        positions_converted1[-1][1],
        positions_converted1[-1][2],
        positions_converted1[-1][3],
        positions_converted1[-1][4],
        positions_converted1[-1][5],
        velocity,
        acceleration,
        0,
    ]
)
rtde_c.moveL(path)

time.sleep(count / 150)

path_2 = []
for i in range(len(positions_converted2) - 1):
    path_2.append(
        [
            positions_converted2[i][0],
            positions_converted2[i][1],
            positions_converted2[i][2],
            positions_converted2[i][3],
            positions_converted2[i][4],
            positions_converted2[i][5],
            velocity,
            acceleration,
            blend_i,
        ]
    )

path_2.append(
    [
        positions_converted2[-1][0],
        positions_converted2[-1][1],
        positions_converted2[-1][2],
        positions_converted2[-1][3],
        positions_converted2[-1][4],
        positions_converted2[-1][5],
        velocity,
        acceleration,
        blend_3,
    ]
)

rtde_c.moveL(path_2)

rtde_c.stopScript()

time.sleep(1)
gripper.close()

In [ ]:
rtde_c.moveL(move_to_pouring_pos, 0.1, 0.1)
rtde_c.moveL(pick_up_pos_4, 0.1, 0.1)
time.sleep(1)
gripper.open()

### Measure results, write to excel file and restart